# Models definition and testing

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from enum import IntEnum
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist

def sq_distance(a: np.ndarray, b: np.ndarray) -> float:
  """
  Calculate the squared distance between two vectors.
  
  Parameters:
  a (np.ndarray): First vector.
  b (np.ndarray): Second vector.
  
  Returns:
  float: Squared distance between the two vectors.
  """
  return np.sum((a - b) ** 2)

def kernel(X: np.ndarray, Y: np.ndarray, cov: np.ndarray) -> np.ndarray:
  """
  Calculate the kernel values between two sets of vectors using a Gaussian kernel.
  
  Parameters:
  X (np.ndarray): First set of vectors (n_samples_X, n_features).
  Y (np.ndarray): Second set of vectors (n_samples_Y, n_features).
  cov (np.ndarray): Covariance matrix for the Gaussian kernel.
  
  Returns:
  np.ndarray: Kernel values between the two sets of vectors (n_samples_X, n_samples_Y).
  """
  n = X.shape[1]
  factor = 1 / (np.sqrt((2 * np.pi) ** n * np.linalg.det(cov)))
  inv_cov = np.linalg.inv(cov)
  diff = X[:, None, :] - Y[None, :, :]
  exponent = -0.5 * np.einsum('ijk,kl,ijl->ij', diff, inv_cov, diff)
  return factor * np.exp(exponent)

def distance_matrix(X: np.ndarray) -> np.ndarray:
  """
  Compute the squared distance matrix for a set of points.
  
  Parameters:
  X (np.ndarray): Array of points.
  
  Returns:
  np.ndarray: Squared distance matrix.
  """
  return cdist(X, X, metric='sqeuclidean')
                    
class Adjacency(IntEnum):
  NOT_ADJACENT = 0,
  GABRIEL_EDGE = 1,
  SUPPORT_EDGE = 2

def gabriel_graph(distance_matrix: np.ndarray) -> np.ndarray:
  """
  Create a Gabriel graph from a distance matrix using the correct condition.

  Let E be an possible edge between points xi and xj.
  E ∈ G <-> ∀k ∈ V \\ {xi, xj} : d^2(xi, xj) ≤ d^2(xi, xk) + d^2(xj, xk)

  Parameters:
  distance_matrix (np.ndarray): Squared distance matrix.

  Returns:
  np.ndarray: Adjacency matrix of the Gabriel graph.
  """
  n = distance_matrix.shape[0]
  adjacency_matrix = np.zeros((n, n), dtype=int)

  for i in range(n):
    for j in range(i + 1, n):
      d_ij = distance_matrix[i, j]
      d_ik = distance_matrix[i, :]
      d_jk = distance_matrix[j, :]
      valid = np.all((d_ik + d_jk) >= d_ij)
      if valid:
        adjacency_matrix[i, j] = Adjacency.GABRIEL_EDGE
        adjacency_matrix[j, i] = Adjacency.GABRIEL_EDGE

  return adjacency_matrix

def support_graph(gabriel_graph: np.ndarray, y: np.ndarray) -> np.ndarray:
  """
  Create a support graph from a Gabriel graph and the vector of labels.

  An edge is a support edge if its vertices belong to different classes.

  Parameters:
  gabriel_graph (np.ndarray): Adjacency matrix of the Gabriel graph.
  y (np.ndarray): Vector of labels.

  Returns:
  np.ndarray: Adjacency matrix of the support graph with enum values.
  """
  n = gabriel_graph.shape[0]
  support_graph = np.zeros((n, n), dtype=int)

  gabriel_edges = gabriel_graph > 0

  different_labels = y[:, None] != y[None, :]

  support_graph[gabriel_edges & different_labels] = Adjacency.SUPPORT_EDGE

  support_graph[gabriel_edges & ~different_labels] = Adjacency.GABRIEL_EDGE

  return support_graph

def plot_graph(X: np.ndarray, graph: np.ndarray, y: np.ndarray, type: str='gabriel') -> None:
  """
  Plot a graph with vertices colored based on labels and edges colored based on their type.

  Parameters:
  X (np.ndarray): Coordinates of the vertices.
  graph (np.ndarray): Adjacency matrix of the graph.
  y (np.ndarray): Labels for the vertices.
  type (str): Type of graph ('gabriel' or 'support').
  """
  G = nx.Graph()
  pos = {i: (X[i, 0], X[i, 1]) for i in range(len(X))}

  # Add nodes with colors based on labels
  for i in range(len(X)):
    G.add_node(i, color='blue' if y[i] == 0 else 'orange')

  # Add edges with colors based on edge type
  for i in range(len(graph)):
    for j in range(i + 1, len(graph)):
      if graph[i, j] == Adjacency.GABRIEL_EDGE:
        if type == 'gabriel':
          G.add_edge(i, j, color='black')
      elif graph[i, j] == Adjacency.SUPPORT_EDGE:
        if type == 'support':
          G.add_edge(i, j, color='red')

  # Extract node and edge colors
  node_colors = [data['color'] for _, data in G.nodes(data=True)]
  edge_colors = [data['color'] for _, _, data in G.edges(data=True)]

  # Plot the graph
  nx.draw(G, pos, node_color=node_colors, edge_color=edge_colors, node_size=10)
  plt.show()

# Generate sample data
X, y = make_moons(n_samples=300, noise=0.25)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, stratify=y)

# Plot the data
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title("Generated Data")
plt.show()

# Compute distance matrix and Gabriel graph
dm = distance_matrix(X_train)
gg = gabriel_graph(dm)

# Compute support graph
sg = support_graph(gg, y_train)

# Plot the Gabriel graph
plt.title("Gabriel Graph")
plot_graph(X_train, gg, y_train, type='gabriel')

# Plot the support graph
plt.title("Support Graph")
plot_graph(X_train, sg, y_train, type='support')

In [ ]:
from typing import Self
from sklearn.base import BaseEstimator, ClassifierMixin

class NN_CLAS(BaseEstimator, ClassifierMixin):
  def _quality_array(self, support_graph: np.ndarray) -> np.ndarray:
    """
    Calculate the quality array for the set.

    q_xi = Ĝ(xi) / G(xi)
    where G(xi) is the quantity of edges that link to it
    and Ĝ(xi) is the quantity of edges that link to it and are not support edges.

    Parameters:
    support_graph (np.ndarray): Adjacency matrix of the support graph.

    Returns:
    np.ndarray: Quality array where each element represents the quality of the corresponding vertex.
    """
    G_xi = np.sum(support_graph != Adjacency.NOT_ADJACENT, axis=1)

    G_xi_hat = np.sum(support_graph == Adjacency.GABRIEL_EDGE, axis=1)

    quality = np.divide(G_xi_hat, G_xi, out=np.zeros_like(G_xi, dtype=float), where=G_xi > 0)

    return quality

  def _filter(self, X: np.ndarray, y: np.ndarray,
              distance_matrix: np.ndarray, quality: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Filter the dataset based on the quality array.
    Vertices with quality less than their class's average quality are removed.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.
    distance_matrix (np.ndarray): Squared distance matrix.
    quality (np.ndarray): Quality array.

    Returns:
    tuple[np.ndarray, np.ndarray, np.ndarray]: Filtered coordinates, labels, and distance matrix.
    """
    if len(y) == 0 or len(quality) == 0:
      return np.array([]), np.array([]), np.array([[]])

    _, inverse_indices = np.unique(y, return_inverse=True)
    class_avg = np.bincount(inverse_indices, weights=quality) / np.bincount(inverse_indices)

    mask = quality >= class_avg[inverse_indices]

    kept_indices = np.where(mask)[0]
    X_filtered = X[kept_indices]
    y_filtered = y[kept_indices]
    dm_filtered = distance_matrix[np.ix_(kept_indices, kept_indices)]

    return X_filtered, y_filtered, dm_filtered
  
  def _experts(self, X: np.ndarray, y: np.ndarray, support_graph: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Get the experts from the support graph.

    A vertex is an expert if and only if it is linked to a support edge.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.
    support_graph (np.ndarray): Adjacency matrix of the support graph.

    Returns:
    np.ndarray: Array of experts coordinates and labels.
    """

    expert_mask = np.any(support_graph == Adjacency.SUPPORT_EDGE, axis=1)

    expert_coords = X[expert_mask]
    expert_labels = y[expert_mask]

    return expert_coords, expert_labels

  def fit(self, X: np.ndarray, y: np.ndarray) -> Self:
    """
    Fit the model to the data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.

    Returns:
    Self: Fitted model.
    """
    self.X_train = X
    self.classes_ = y
    self.dm = distance_matrix(X)
    self.gg = gabriel_graph(self.dm)
    self.sg = support_graph(self.gg, y)
    self.quality = self._quality_array(self.sg)
    self.X_filtered, self.y_filtered, self.dm = self._filter(X, y, self.dm, self.quality)
    self.gg = gabriel_graph(self.dm)
    self.sg = support_graph(self.gg, self.y_filtered)
    self.expert_coords, self.expert_labels = self._experts(self.X_filtered, self.y_filtered, self.sg)

    return self

  def predict(self, X: np.ndarray) -> np.ndarray:
    """
    Predict the labels for the given data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices to predict.

    Returns:
    np.ndarray: Predicted labels.
    """

    distances = cdist(X, self.expert_coords, metric='sqeuclidean')
    
    nearest_expert_indices = np.argmin(distances, axis=1)
    
    return self.expert_labels[nearest_expert_indices]

nn_clas = NN_CLAS()

nn_clas.fit(X_train, y_train)

# Plot the filtered Support graph
plt.title("Filtered Support Graph")
plot_graph(nn_clas.X_filtered, nn_clas.sg, nn_clas.y_filtered, type='support')


In [ ]:
from sklearn.metrics import accuracy_score

y_pred_nnclas = nn_clas.predict(X_test)

def check(X_train: np.ndarray, y_train: np.ndarray,
          X_test: np.ndarray, y_test: np.ndarray,
          y_pred: np.ndarray,
          class_name: str) -> None:
  """
  Check the predictions and print the results.

  Parameters:
  X_train (np.ndarray): Training data.
  y_train (np.ndarray): Training labels.
  X_test (np.ndarray): Test data.
  y_test (np.ndarray): Test labels.
  y_pred (np.ndarray): Predicted labels.
  """

  acc = accuracy_score(y_test, y_pred)

  # Plot train, test, and experts
  plt.figure()

  # Plot train data
  plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolor='black', alpha=0.5, s=25)

  # Plot test data
  correct = y_pred == y_test
  plt.scatter(X_test[correct, 0], X_test[correct, 1], c=y_pred[correct], edgecolor='green', marker='D')
  plt.scatter(X_test[~correct, 0], X_test[~correct, 1], c=y_pred[~correct], edgecolor='red', marker='D')

  plt.title(f'{class_name} - accuracy: {acc:.3f}')
  plt.show()

check(X_train, y_train, X_test, y_test, y_pred_nnclas, "NN_CLAS")

In [ ]:
K = 5

class KNN(BaseEstimator, ClassifierMixin):
  def __init__(self, k: int = 3) -> None:
    """
    Initialize the KNN classifier.

    Parameters:
    k (int): Number of neighbors to consider.
    """
    self.k = k

  def fit(self, X: np.ndarray, y: np.ndarray) -> Self:
    """
    Fit the model to the data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.

    Returns:
    Self: Fitted model.
    """
    self.X_train = X

    unique_values = np.unique(y)
    self.label_map = {unique_values[0]: -1, unique_values[1]: 1}
    self.inverse_label_map = {-1: unique_values[0], 1: unique_values[1]}

    self.classes_ = np.array([self.label_map[label] for label in y.flatten()])

    self.cov = np.cov(X, rowvar=False)

    return self

  def predict(self, X: np.ndarray) -> np.ndarray:
    """
    Predict the labels for the given data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices to predict.

    Returns:
    np.ndarray: Predicted labels.
    """

    predictions = []
    for x in X:
      distances = np.array([sq_distance(x, x_train) for x_train in self.X_train])
      nearest_indices = np.argsort(distances)[:self.k]
      scores = np.array([
        self.classes_[idx] * kernel(x.reshape(1, -1), self.X_train[idx].reshape(1, -1), self.cov)[0, 0]
        for idx in nearest_indices
      ])
      predictions.append(self.inverse_label_map[1 if scores.sum() > 0 else -1])
    return np.array(predictions)
  
  def likelihood_score(self, X: np.ndarray, y: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculate the likelihood score for the given data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.

    Returns:
    tuple[np.ndarray, np.ndarray]: Likelihood scores for each class.
    """
    Q0 = np.zeros(X.shape[0])
    Q1 = np.zeros(X.shape[0])

    self.fit(X, y)

    distances = cdist(X, self.X_train, metric='sqeuclidean')
    nearest_indices = np.argsort(distances, axis=1)[:, :self.k]

    for i in range(X.shape[0]):
      Q0[i] = sum(
        kernel(X[i].reshape(1, -1), self.X_train[idx].reshape(1, -1), self.cov)[0, 0]
        for idx in nearest_indices[i] if self.classes_[idx] == -1
      )
      Q1[i] = sum(
        kernel(X[i].reshape(1, -1), self.X_train[idx].reshape(1, -1), self.cov)[0, 0]
        for idx in nearest_indices[i] if self.classes_[idx] == 1
      )

    return Q0, Q1

knn = KNN(k=K)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

check(X_train, y_train, X_test, y_test, y_pred_knn, "KNN")

In [ ]:
class KNN_CLAS(BaseEstimator, ClassifierMixin):
  def __init__(self, k: int = 3) -> None:
    """
    Initialize the KNN classifier.

    Parameters:
    k (int): Number of neighbors to consider.
    """
    self.k = k

  def _experts(self, X: np.ndarray, y: np.ndarray, support_graph: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Get the experts from the support graph.

    A vertex is an expert if and only if it is linked to a support edge.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.
    support_graph (np.ndarray): Adjacency matrix of the support graph.

    Returns:
    np.ndarray: Array of experts coordinates and labels.
    """

    expert_mask = np.any(support_graph == Adjacency.SUPPORT_EDGE, axis=1)

    expert_coords = X[expert_mask]
    expert_labels = y[expert_mask]

    return expert_coords, expert_labels

  def fit(self, X: np.ndarray, y: np.ndarray) -> Self:
    """
    Fit the model to the data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.

    Returns:
    Self: Fitted model.
    """
    self.X_train = X

    unique_values = np.unique(y)
    self.label_map = {unique_values[0]: -1, unique_values[1]: 1}
    self.inverse_label_map = {-1: unique_values[0], 1: unique_values[1]}

    self.classes_ = np.array([self.label_map[label] for label in y.flatten()])

    self.cov = np.cov(X, rowvar=False)

    self.dm = distance_matrix(X)
    self.gg = gabriel_graph(self.dm)
    self.sg = support_graph(self.gg, y)

    self.expert_coords, self.expert_labels = self._experts(X, y, self.sg)
    self.expert_labels = np.array([self.label_map[label] for label in self.expert_labels])

    return self
  
  def predict(self, X: np.ndarray) -> np.ndarray:
    """
    Predict the labels for the given data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices to predict.

    Returns:
    np.ndarray: Predicted labels.
    """
    distances = cdist(X, self.expert_coords, metric='sqeuclidean')
    nearest_indices = np.argsort(distances, axis=1)[:, :self.k]
    scores = np.array([
      np.sum([
        self.expert_labels[idx] * kernel(X[i].reshape(1, -1), self.expert_coords[idx].reshape(1, -1), self.cov)[0, 0]
        for idx in nearest_indices[i]
      ])
      for i in range(X.shape[0])
    ])
    return np.array([self.inverse_label_map[1 if score > 0 else -1] for score in scores])
  
  def likelihood_score(self, X: np.ndarray, y: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculate the likelihood score for the given data.

    Parameters:
    X (np.ndarray): Coordinates of the vertices.
    y (np.ndarray): Labels for the vertices.

    Returns:
    tuple[np.ndarray, np.ndarray]: Likelihood scores for each class.
    """
    Q0 = np.zeros(X.shape[0])
    Q1 = np.zeros(X.shape[0])

    self.fit(X, y)

    distances = cdist(X, self.expert_coords, metric='sqeuclidean')
    nearest_indices = np.argsort(distances, axis=1)[:, :self.k]

    for i in range(X.shape[0]):
      Q0[i] = sum(
        kernel(X[i].reshape(1, -1), self.expert_coords[idx].reshape(1, -1), self.cov)[0, 0]
        for idx in nearest_indices[i] if self.expert_labels[idx] == -1
      )
      Q1[i] = sum(
        kernel(X[i].reshape(1, -1), self.expert_coords[idx].reshape(1, -1), self.cov)[0, 0]
        for idx in nearest_indices[i] if self.expert_labels[idx] == 1
      )

    return Q0, Q1
  
knn_clas = KNN_CLAS(k=K)
knn_clas.fit(X_train, y_train)
y_pred_knn_clas = knn_clas.predict(X_test)

check(X_train, y_train, X_test, y_test, y_pred_knn_clas, "KNN_CLAS")

In [ ]:
knn_Q0, knn_Q1 = knn.likelihood_score(X, y)
knn_clas_Q0, knn_clas_Q1 = knn_clas.likelihood_score(X, y)

unique_labels = np.unique(y)
y01 = np.array([0 if label == unique_labels[0] else 1 for label in y])

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(knn_Q0, knn_Q1, c=y01)
plt.title('KNN')
plt.subplot(1, 2, 2)
plt.scatter(knn_clas_Q0, knn_clas_Q1, c=y01)
plt.title('KNN_CLAS')
plt.tight_layout()
plt.show()

# Real data loading

In [ ]:
import os
import numpy as np

sets_directory = './sets/'
datasets = {}

for file_name in os.listdir(sets_directory):
  if file_name.endswith('.npz'):
    file_path = os.path.join(sets_directory, file_name)
    data = np.load(file_path, allow_pickle=True)
    set_name = file_name.split('.')[0]
    datasets[set_name] = {'X': data['X'], 'y': data['y']}
    print(f"Loaded {set_name}: X shape {data['X'].shape}, y shape {data['y'].shape}")

# Test classifiers on real data

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

SET = 'haberman_survival'
K = 5

X = datasets[SET]['X']
y = datasets[SET]['y'].ravel()

kf = KFold(n_splits=30, shuffle=True)

model_constructors = {
  "KNN": lambda: KNN(k=K),
  "KNN_CLAS": lambda: KNN_CLAS(k=K)
}

model_names = list(model_constructors.keys())
model_metrics = {name: {'accuracy': [], 'precision': [], 'recall': [], 'f1': []} for name in model_names}

for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  for name in model_names:
    model = model_constructors[name]()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    model_metrics[name]['accuracy'].append(accuracy_score(y_test, y_pred))
    model_metrics[name]['precision'].append(precision_score(y_test, y_pred, average='weighted', zero_division=0))
    model_metrics[name]['recall'].append(recall_score(y_test, y_pred, average='weighted', zero_division=0))
    model_metrics[name]['f1'].append(f1_score(y_test, y_pred, average='weighted', zero_division=0))

In [ ]:
from scipy.stats import wilcoxon

metrics = ['accuracy', 'precision', 'recall', 'f1']

num_comparisons = len(metrics)
bonferroni_alpha = 0.05 / num_comparisons

print("\nStatistical Tests (Wilcoxon with Bonferroni Correction):")
for metric in metrics:
  print(f"\nMetric: {metric}")
  data1 = model_metrics["KNN"][metric]
  data2 = model_metrics["KNN_CLAS"][metric]
  stat, p_value = wilcoxon(data1, data2)
  significance = "SIGNIFICANT" if p_value < bonferroni_alpha else "NOT significant"
  print(f"p = {p_value:.4f} ({significance})")

In [ ]:
from sklearn.utils import resample

def bootstrap_precision(model1, model2, X, y, n_bootstraps=100):
  precision_diff = []
  for _ in range(n_bootstraps):
    # Resample with replacement
    X_resampled, y_resampled = resample(X, y, replace=True)
    
    # Train and evaluate both models
    m1 = model1().fit(X_resampled, y_resampled)
    y_pred1 = m1.predict(X_resampled)
    p1 = precision_score(y_resampled, y_pred1, average='weighted', zero_division=0)
    
    m2 = model2().fit(X_resampled, y_resampled)
    y_pred2 = m2.predict(X_resampled)
    p2 = precision_score(y_resampled, y_pred2, average='weighted', zero_division=0)
    
    precision_diff.append(p1 - p2)
    
  # Compute 95% confidence interval
  ci_low = np.percentile(precision_diff, 2.5)
  ci_high = np.percentile(precision_diff, 97.5)
  return ci_low, ci_high

# Run for KNN_CLAS vs KNN
ci_low, ci_high = bootstrap_precision(
  lambda: KNN_CLAS(k=K), 
  lambda: KNN(k=K), 
  X, y, n_bootstraps=100
)
print(f"\nBootstrap 95% CI for Precision: [{ci_low:.3f}, {ci_high:.3f}]")

In [ ]:
from scipy.stats import permutation_test

def statistic(x, y):
  return np.mean(x) - np.mean(y)

knnclas_precision = model_metrics['KNN_CLAS']['precision']
knn_precision = model_metrics['KNN']['precision']

res = permutation_test((knnclas_precision, knn_precision), statistic, n_resamples=10000)
print(f"Permutation test p-value: {res.pvalue:.4f}")

In [ ]:
knn = KNN(k=K)
knn_clas = KNN_CLAS(k=K)

knn_Q0, knn_Q1 = knn.likelihood_score(X, y)
knn_clas_Q0, knn_clas_Q1 = knn_clas.likelihood_score(X, y)

unique_labels = np.unique(y)
y01 = np.array([0 if label == unique_labels[0] else 1 for label in y])

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(knn_Q0, knn_Q1, c=y01)
plt.title('KNN')
plt.subplot(1, 2, 2)
plt.scatter(knn_clas_Q0, knn_clas_Q1, c=y01)
plt.title('KNN_CLAS')
plt.tight_layout()
plt.show()

In [ ]:
# Compute spatial metrics to compare distributions
knn_scores = np.column_stack((knn_Q0, knn_Q1))
knn_clas_scores = np.column_stack((knn_clas_Q0, knn_clas_Q1))

# Centroid Distance
centroid_knn = np.mean(knn_scores, axis=0)
centroid_clas = np.mean(knn_clas_scores, axis=0)
centroid_distance = np.linalg.norm(centroid_knn - centroid_clas)

# Mahalanobis Distance (using pooled covariance)
n1, n2 = len(knn_scores), len(knn_clas_scores)
cov_knn = np.cov(knn_scores, rowvar=False)
cov_clas = np.cov(knn_clas_scores, rowvar=False)
pooled_cov = ((n1 - 1)*cov_knn + (n2 - 1)*cov_clas) / (n1 + n2 - 2)
inv_pooled_cov = np.linalg.pinv(pooled_cov)  # Using pseudo-inverse for stability
delta_centroid = centroid_knn - centroid_clas
mahalanobis_distance = np.sqrt(delta_centroid.T @ inv_pooled_cov @ delta_centroid)

# Bhattacharyya Distance
sigma = (cov_knn + cov_clas) / 2
mu_diff = centroid_knn - centroid_clas
term1 = 0.125 * (mu_diff.T @ np.linalg.pinv(sigma) @ mu_diff)
det_sigma = np.linalg.det(sigma + 1e-6 * np.eye(sigma.shape[0]))  # Regularize to avoid singular
det_sigma1 = np.linalg.det(cov_knn + 1e-6 * np.eye(cov_knn.shape[0]))
det_sigma2 = np.linalg.det(cov_clas + 1e-6 * np.eye(cov_clas.shape[0]))
term2 = 0.5 * np.log(det_sigma / np.sqrt(det_sigma1 * det_sigma2))
bhattacharyya = term1 + term2

import ot

a = np.ones(n1) / n1
b = np.ones(n2) / n2
M = ot.dist(knn_scores, knn_clas_scores)
emd = ot.emd2(a, b, M)

# Hotelling's T-squared test (paired)
from scipy.stats import chi2
diffs = knn_scores - knn_clas_scores
n = diffs.shape[0]
mean_diff = np.mean(diffs, axis=0)
cov_diff = np.cov(diffs, rowvar=False)
inv_cov_diff = np.linalg.pinv(cov_diff)
T2 = n * mean_diff.T @ inv_cov_diff @ mean_diff
p_hotelling = 1 - chi2.cdf(T2, df=2)

# Kolmogorov-Smirnov tests for each dimension
from scipy.stats import ks_2samp
ks_q0 = ks_2samp(knn_Q0, knn_clas_Q0)
ks_q1 = ks_2samp(knn_Q1, knn_clas_Q1)

print("\nSpatial Distribution Metrics:")
print(f"Centroid Distance: {centroid_distance:.4f}")
print(f"Mahalanobis Distance: {mahalanobis_distance:.4f}")
print(f"Bhattacharyya Distance: {bhattacharyya:.4f}")
print(f"Earth Mover's Distance (EMD): {emd:.4f}")
print(f"Hotelling's T-squared p-value: {p_hotelling:.4f}")
print(f"KS Test Q0: statistic={ks_q0.statistic:.4f}, p={ks_q0.pvalue:.4f}")
print(f"KS Test Q1: statistic={ks_q1.statistic:.4f}, p={ks_q1.pvalue:.4f}")

In [ ]:
from tqdm import tqdm
import json
import time
import sys

def input_space_statistical_validation(
  datasets: dict,
  set_name: str,
  k: int = 5,
  n_splits: int = 30,
  n_bootstraps: int = 100,
  n_permutations: int = 10000,
  timeout: float = 3600  # 1 hour timeout
) -> dict:
  """
  Perform statistical validation with progress tracking and safety measures.
  """
  start_time = time.time()
  print(f"\n{'='*40}\nProcessing dataset: {set_name}\n{'='*40}")

  def check_timeout():
    if time.time() - start_time > timeout:
      raise TimeoutError(f"Execution exceeded {timeout} second timeout")

  try:
    # Load dataset
    X = datasets[set_name]['X']
    y = datasets[set_name]['y'].ravel()
    
    # Initialize results structure
    results = {
      'dataset': set_name,
      'start_time': time.strftime('%Y-%m-%d %H:%M:%S'),
      'model_metrics': {'KNN': {}, 'KNN_CLAS': {}},
      'statistical_tests': {},
      'completed': False,
      'error': None
    }

    # Phase 1: Cross-validation
    print("\n[Phase 1/3] Cross-validation")
    model_constructors = {
      "KNN": KNN(k=k),
      "KNN_CLAS": KNN_CLAS(k=k)
    }
    kf = KFold(n_splits=n_splits, shuffle=True)
    
    # Initialize metrics storage with progress tracking
    with tqdm(total=n_splits*2, desc="CV Folds") as pbar:
      for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
        check_timeout()
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        for model_name, model in model_constructors.items():
          try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            # Store metrics
            for metric, fn in [('accuracy', accuracy_score),
                     ('precision', partial(precision_score, average='weighted', zero_division=0)),
                     ('recall', partial(recall_score, average='weighted', zero_division=0)),
                     ('f1', partial(f1_score, average='weighted', zero_division=0))]:
              if fold == 1:
                results['model_metrics'][model_name][metric] = []
              results['model_metrics'][model_name][metric].append(fn(y_test, y_pred))
              
            pbar.update(1)
            pbar.set_postfix({'fold': fold, 'model': model_name})
            
          except Exception as e:
            print(f"\nError in {model_name} fold {fold}: {str(e)}")
            raise

    # Phase 2: Statistical Tests
    print("\n[Phase 2/3] Statistical Analysis")
    # Wilcoxon tests
    print("  Running Wilcoxon tests...")
    bonferroni_alpha = 0.05 / 4
    results['statistical_tests']['wilcoxon'] = {}
    for metric in tqdm(['accuracy', 'precision', 'recall', 'f1'], desc="Metrics"):
      check_timeout()
      knn_data = results['model_metrics']['KNN'][metric]
      knn_clas_data = results['model_metrics']['KNN_CLAS'][metric]
      stat, p_value = wilcoxon(knn_data, knn_clas_data)
      results['statistical_tests']['wilcoxon'][metric] = {
        'p_value': float(p_value),
        'significant': p_value < bonferroni_alpha
      }

    # Bootstrap CI
    print("  Bootstrapping...")
    boot_diffs = []
    for _ in tqdm(range(n_bootstraps), desc="Bootstrap Iterations"):
      check_timeout()
      X_res, y_res = resample(X, y, replace=True)
      knn = KNN(k=k).fit(X_res, y_res)
      knn_clas = KNN_CLAS(k=k).fit(X_res, y_res)
      boot_diffs.append(
        precision_score(y_res, knn_clas.predict(X_res), average='weighted', zero_division=0) -
        precision_score(y_res, knn.predict(X_res), average='weighted', zero_division=0)
      )
    results['statistical_tests']['bootstrap_precision_ci'] = [
      float(np.percentile(boot_diffs, 2.5)),
      float(np.percentile(boot_diffs, 97.5))
    ]

    # Permutation test
    print("  Permutation testing...")
    knn_prec = results['model_metrics']['KNN']['precision']
    knn_clas_prec = results['model_metrics']['KNN_CLAS']['precision']
    with tqdm(total=n_permutations, desc="Permutations") as pbar:
      def callback(x, y):
        pbar.update(1)
        check_timeout()
        
      perm_result = permutation_test(
        (knn_clas_prec, knn_prec), 
        lambda x, y: np.mean(x) - np.mean(y),
        n_resamples=n_permutations,
        callback=callback
      )
    results['statistical_tests']['permutation_p_value'] = float(perm_result.pvalue)

    # Finalize
    results['end_time'] = time.strftime('%Y-%m-%d %H:%M:%S')
    results['duration'] = time.time() - start_time
    results['completed'] = True
    print(f"\n✅ Completed {set_name} in {results['duration']:.1f}s")

    return results

  except Exception as e:
    results['error'] = str(e)
    results['end_time'] = time.strftime('%Y-%m-%d %H:%M:%S')
    results['duration'] = time.time() - start_time
    print(f"\n❌ Failed {set_name} after {results['duration']:.1f}s: {str(e)}")
    return results
  

from functools import partial

# Process all datasets with progress
final_results = {}
dataset_names = list(datasets.keys())

for idx, dataset_name in enumerate(dataset_names, 1):
  print(f"\n📂 Processing dataset {idx}/{len(dataset_names)}")
  final_results[dataset_name] = input_space_statistical_validation(
    datasets, dataset_name, k=5
  )
  
  # Early exit if 3 consecutive failures
  failures = sum(1 for r in final_results.values() if not r['completed'])
  if failures >= 3:
    print("⚠️ Aborting: 3 consecutive failures")
    break

# Save results
with open('results.json', 'w') as f:
  json.dump(final_results, f, indent=2, default=str)

